# !!!!!!! CLEAR OUTPUT BEFORE COMMIT !!!!!!
TODO:
- Try different models
  - RandomForest - Done
  - XGBoost
  - LightGBM  
- Check for outliers and remove?
- Apply inflation to sale price?
- Add heat map of correlations
- Drop Garage Area (same information as Garage Cars)
- Same Data Points (Keep vs Drop):
  - Garage Cars vs Garage Area
  - 'GrLivArea' vs 'TotRmsAbvGrd'
- Drop Features > 15% missing
  - Drop BsmntX
  - Drop MasnVnrX
  - Drop GarageX Not Cars
  - Drop 1 record missing electical
- saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis]);
  - Look at two outlier values > 7 - Drop?
- Scatter plot SalePrice vs GrLivArea
  - Drop two outlier observations
- Log on GrLivArea

In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def one_hot_encode(df, col_name_list):
    #for col_name in col_name_list:
    #    df = pd.concat((df, pd.get_dummies(df[col_name], prefix=col_name)), axis='columns')
    df.drop(columns=col_name_list, inplace=True)
    return df

def a_f_ohe(df, column_name, one_hot_encode_list):
    if not one_hot_encode_list:
        one_hot_encode_list = []
    #df[column_name] = column_name + df[column_name].astype(str)
    if not column_name in one_hot_encode_list:
        one_hot_encode_list = one_hot_encode_list + [column_name]
    return one_hot_encode_list

def calculate_ages(df):
    df['SaleDate'] = df['YrSold'].astype(str) + '-' + df['MoSold'].astype(str)
    df['SaleDate'] = pd.to_datetime(df['SaleDate'], format='%Y-%m')
    
    # YrSold
    df['SaleAge'] = (df['SaleDate'].dt.year + df['SaleDate'].dt.month / 12) - \
        (df['YrSold'].astype(float) + df['MoSold'].astype(float) / 12)
    del df['YrSold']
    
    # YearBuilt
    df['Age'] = df['SaleDate'].dt.year - df['YearBuilt']
    del df['YearBuilt']
    
    # YearRemodAdd
    df['RemodelAge'] = df['SaleDate'].dt.year - df['YearRemodAdd']
    del df['YearRemodAdd']
    
    # GarageYrBlt
    df['GarageAge'] = df['SaleDate'].dt.year - df['GarageYrBlt']
    del df['GarageYrBlt']
    
    # we are done using this field and it is not numeric, which causes issues with sklearn
    del df['SaleDate']
    return df

def impute_lot_frontage(df):
    avg_ratio = (df['LotFrontage'] / df['LotArea']).mean()
    df['LotFrontage'] = df['LotFrontage'].fillna(df['LotArea'] * avg_ratio)
    return df

def process_df(df, ohe_list):
    df['CentralAir'] = df['CentralAir'].map({'Y':1,'N':0}).astype(int)
    
    mask = df['MasVnrType'] == 'None'
    df.loc[mask, 'MasVnrType'] = np.NAN
    
    mask = df['MasVnrType'].isnull()
    df.loc[mask, 'MasVnrArea'] = 0.0
    df['MasVnrArea'].fillna(0.0, inplace=True)

    mask = df['GarageYrBlt'].isnull()
    df.loc[mask, 'GarageYrBlt'] = df.loc[mask, 'YearBuilt']
    
    df = calculate_ages(df)
    df = impute_lot_frontage(df)
    df = one_hot_encode(df, ohe_list)
    if 'SalePrice' in df.columns.values:
        df['SalePrice'] = np.log(df['SalePrice'])
    return df

In [ ]:
df = pd.read_csv(os.path.join('data','train.csv'))

In [ ]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
df.head()

How many records are we dealing with?

In [ ]:
len(df.index)

Because of the small size of the training and testing datasets we don't need to worry about converting data types to smaller sizes to conserve space.

What data types are we dealing with?

In [ ]:
df.dtypes

Let's take a look at the different features and get some simple summary statistics

In [ ]:
df['1stFlrSF'].describe()

In [ ]:
df['MSSubClass'].value_counts()

Which fields have null values?

In [ ]:
df_nulls = df.copy().isnull().sum()
df_nulls = df_nulls.to_frame().rename(columns={0:'num_nulls'})
df_nulls['total_records'] = len(df.index)
df_nulls['pct_null'] = df_nulls['num_nulls'] / df_nulls['total_records'] * 100
df_nulls[df_nulls['num_nulls'] > 0].sort_values(by='pct_null', ascending=False)

Wow, the PoolQC (Pool Quality) is almost always empty. Is this because:
- Most of the houses have no pools?
- How many houses have pools but no quality value?

In [ ]:
df.loc[df['PoolArea'] > 0, ['PoolQC','PoolArea']]

There are no pools without a quality rating.

In [ ]:
df.loc[~df['PoolQC'].isnull(), ['PoolQC','PoolArea']]

This shows us that only 7 houses had a pool and no pool quality records exist where there is no pool (pool area = 0)

Pools normally represent a small amount of houses so it's important to keep this around rather than attempting to just drop it since there are so few records.

To handle these we can One Hot Encode (pd.get_dummies) for the pool quality values. Any house without a pool will have 0 for all the quality measurements to work around the null values in the pool quality feature.

If we're doing something similar to the PoolQC we should explicitly show it like we did with PoolQC

In [ ]:
ohe_list = [] #['PoolQC','Alley','Fence','MiscFeature']

**MSSubClass:** Identifies the type of dwelling involved in the sale.

In [ ]:
# Prefix the categories so they make a bit more sense when we OHE them.
# Initially it's imported as a number so we'll force it to be a string
ohe_list = a_f_ohe(df, 'MSSubClass', ohe_list)

**MSZoning:** Identifies the general zoning classification of the sale.

In [ ]:
ohe_list = a_f_ohe(df, 'MSZoning', ohe_list)

**LotFrontage:** Linear feet of street connected to property

Nothing to do here. This is a simple number. Perhaps scale?

**Street:** Type of road access to property

In [ ]:
ohe_list = a_f_ohe(df, 'Street', ohe_list)

**Alley:** Type of alley access to property

In [ ]:
ohe_list = a_f_ohe(df, 'Alley', ohe_list)

**LotShape:** General shape of property

In [ ]:
ohe_list = a_f_ohe(df, 'LotShape', ohe_list)

**LandContour:** Flatness of the property

In [ ]:
ohe_list = a_f_ohe(df, 'LandContour', ohe_list)

**Utilities:** Type of utilities available

In [ ]:
ohe_list = a_f_ohe(df, 'Utilities', ohe_list)

**LotConfig:** Lot configuration

In [ ]:
ohe_list = a_f_ohe(df, 'LotConfig', ohe_list)

**LandSlope:** Slope of property

In [ ]:
ohe_list = a_f_ohe(df, 'LandSlope', ohe_list)

**Neighborhood:** Physical locations within Ames city limits

In [ ]:
ohe_list = a_f_ohe(df, 'Neighborhood', ohe_list)

**Condition1:** Proximity to various conditions

Are these mutually exclusive values? They appear so.

In [ ]:
df['Condition1'].value_counts()

In [ ]:
ohe_list = a_f_ohe(df, 'Condition1', ohe_list)

**Condition2:** Proximity to various conditions (if more than one is present)

In [ ]:
df['Condition2'].value_counts()

In [ ]:
ohe_list = a_f_ohe(df, 'Condition2', ohe_list)

**BldgType:** Type of dwelling

In [ ]:
ohe_list = a_f_ohe(df, 'BldgType', ohe_list)

**HouseStyle:** Style of dwelling

In [ ]:
ohe_list = a_f_ohe(df, 'HouseStyle', ohe_list)

**OverallQual:** Rates the overall material and finish of the house

Rating between 1 and 10 (Very Poor and Very Excellent) - Since this is numeric and has a natural relationship we'll leave alone.

**OverallCond:** Rates the overall condition of the house

Rating between 1 and 10 (Very Poor and Very Excellent) - Since this is numeric and has a natural relationship we'll leave alone.

**YearBuilt:** Original construction date

Convert this to be the ages in full years.

This is done within the calculate_ages to apply it to the train and test sets the same way.

**YearRemodAdd:** Remodel date (same as construction date if no remodeling or additions)

This is the same process as the YearBuilt

**RoofStyle:** Type of roof

In [ ]:
ohe_list = a_f_ohe(df, 'RoofStyle', ohe_list)

**RoofMatl:** Roof material

In [ ]:
ohe_list = a_f_ohe(df, 'RoofMatl', ohe_list)

**Exterior1st:** Exterior covering on house

In [ ]:
ohe_list = a_f_ohe(df, 'Exterior1st', ohe_list)

**Exterior2nd:** Exterior covering on house (if more than one material)

In [ ]:
ohe_list = a_f_ohe(df, 'Exterior2nd', ohe_list)

**MasVnrType:** Masonry veneer type

Convert None values to NaN, OHE the rest

In [ ]:
mask = df['MasVnrType'] == 'None'
df.loc[mask, 'MasVnrType'] = np.NAN
ohe_list = a_f_ohe(df, 'MasVnrType', ohe_list)

**MasVnrArea:** Masonry veneer area in square feet

When MasVnrType is NaN, there a few (5) rows where the area is greater than zero. We are going to assume the area is supposed to be zero when there is no masonry veneer.

In [ ]:
mask = df['MasVnrType'].isnull()
df[mask]['MasVnrArea'].value_counts()

In [ ]:
df.loc[mask, 'MasVnrArea'] = 0.0
df['MasVnrArea'].fillna(0.0, inplace=True)

**ExterQual:** Evaluates the quality of the material on the exterior

In [ ]:
ohe_list = a_f_ohe(df, 'ExterQual', ohe_list)

**ExterQual:** Evaluates the quality of the material on the exterior

In [ ]:
ohe_list = a_f_ohe(df, 'ExterQual', ohe_list)

**ExterCond:** Evaluates the present condition of the material on the exterior

In [ ]:
ohe_list = a_f_ohe(df, 'ExterCond', ohe_list)

**Foundation:** Type of foundation

In [ ]:
ohe_list = a_f_ohe(df, 'Foundation', ohe_list)

**BsmtQual:** Evaluates the height of the basement

In [ ]:
ohe_list = a_f_ohe(df, 'BsmtQual', ohe_list)

**BsmtCond:** Evaluates the general condition of the basement

In [ ]:
ohe_list = a_f_ohe(df, 'BsmtCond', ohe_list)

**BsmtExposure:** Refers to walkout or garden level walls

In [ ]:
ohe_list = a_f_ohe(df, 'BsmtExposure', ohe_list)

**BsmtFinType1:** Rating of basement finished area

In [ ]:
ohe_list = a_f_ohe(df, 'BsmtFinType1', ohe_list)

**BsmtFinSF1:** Type 1 finished square feet
Numeric field. Perhaps Scale?

**BsmtFinType2:** Rating of basement finished area (if multiple types)

In [ ]:
ohe_list = a_f_ohe(df, 'BsmtFinType2', ohe_list)

**BsmtFinSF2:** Type 2 finished square feet

Number - Scale?

**BsmtUnfSF:** Unfinished square feet of basement area

Number - scale?

**TotalBsmtSF:** Total square feet of basement area

Number - Scale?

**Heating:** Type of heating

In [ ]:
ohe_list = a_f_ohe(df, 'Heating', ohe_list)

**HeatingQC:** Heating quality and condition

In [ ]:
ohe_list = a_f_ohe(df, 'HeatingQC', ohe_list)

**CentralAir:** Central air conditioning

Boolean. Convery Y/N to 1/0

In [ ]:
df['CentralAir'] = df['CentralAir'].map({'Y':1,'N':0}).astype(int)

**Electrical:** Electrical system

In [ ]:
ohe_list = a_f_ohe(df, 'Electrical', ohe_list)

**1stFlrSF:** First Floor square feet

**2ndFlrSF:** Second floor square feet
 
**LowQualFinSF:** Low quality finished square feet (all floors)
 
**GrLivArea:** Above grade (ground) living area square feet
 
**BsmtFullBath:** Basement full bathrooms
 
**BsmtHalfBath:** Basement half bathrooms
 
**FullBath:** Full bathrooms above grade
 
**HalfBath:** Half baths above grade
 
**Bedroom:** Bedrooms above grade (does NOT include basement bedrooms)
 
**Kitchen:** Kitchens above grade

All numerical - not much to do here. Probably scale?

**KitchenQual:** Kitchen quality

In [ ]:
ohe_list = a_f_ohe(df, 'KitchenQual', ohe_list)

**TotRmsAbvGrd:** Total rooms above grade (does not include bathrooms)

Number - not much to do here. Scale?

**Functional:** Home functionality (Assume typical unless deductions are warranted)

In [ ]:
ohe_list = a_f_ohe(df, 'Functional', ohe_list)

**Fireplaces:** Number of fireplaces

Number, not much to do. Scale?

**FireplaceQu:** Fireplace quality

In [ ]:
ohe_list = a_f_ohe(df, 'FireplaceQu', ohe_list)

**GarageType:** Garage location

In [ ]:
ohe_list = a_f_ohe(df, 'GarageType', ohe_list)

**GarageYrBlt**: Year garage was built

Year - Convert to Age in years.

This is done within the calculate_ages function to apply to the train and test data sets the same way

In [ ]:
mask = df['GarageYrBlt'].isnull()
df.loc[mask, 'GarageYrBlt'] = df.loc[mask, 'YearBuilt'] 

**GarageFinish:** Interior finish of the garage

In [ ]:
ohe_list = a_f_ohe(df, 'GarageFinish', ohe_list)

**GarageCars:** Size of garage in car capacity
 
**GarageArea:** Size of garage in square feet

Numeric fields - scale?

**GarageQual:** Garage quality

In [ ]:
ohe_list = a_f_ohe(df, 'GarageQual', ohe_list)

**GarageCond:** Garage condition

In [ ]:
ohe_list = a_f_ohe(df, 'GarageCond', ohe_list)

**PavedDrive:** Paved driveway

In [ ]:
ohe_list = a_f_ohe(df, 'PavedDrive', ohe_list)

**WoodDeckSF:** Wood deck area in square feet
 
**OpenPorchSF:** Open porch area in square feet
 
**EnclosedPorch:** Enclosed porch area in square feet
 
**3SsnPorch:** Three season porch area in square feet
 
**ScreenPorch:** Screen porch area in square feet
 
**PoolArea:** Pool area in square feet

Numeric, nothing to do.

**PoolQC:** Pool quality

In [ ]:
ohe_list = a_f_ohe(df, 'PoolQC', ohe_list)

**Fence:** Fence quality

In [ ]:
ohe_list = a_f_ohe(df, 'Fence', ohe_list)

**MiscFeature:** Miscellaneous feature not covered in other categories

In [ ]:
ohe_list = a_f_ohe(df, 'MiscFeature', ohe_list)

**MiscVal:** $Value of miscellaneous feature

Numeric - Nothing to do

**YrSold:** Year Sold (YYYY)

This is done within the calculate_ages function to apply to the train and test datasets the same way.

**MoSold:** Month Sold (MM)

In [ ]:
ohe_list = a_f_ohe(df, 'MoSold', ohe_list)

**SaleType:** Type of sale

In [ ]:
ohe_list = a_f_ohe(df, 'SaleType', ohe_list)

**SaleCondition:** Condition of sale

In [ ]:
ohe_list = a_f_ohe(df, 'SaleCondition', ohe_list)

### Transform / Prepare the DataFrame

In [ ]:
df = calculate_ages(df)
df = one_hot_encode(df, ohe_list)

How many columns do we have?

In [ ]:
len(df.columns)

Will this be too many columns or will our model function well enough as is? We might need to come back and re-evaulate all of the features within our dataset. Perhaps we might need to perform some Principal Component Analysis (PCA) to reduce our feature space.

## Scale the data
Since all of our data elements are now processed we need to scale our data so that every feature is on the same scale and we don't have certain features overly affecting the model.

StandardScaler requires data to not contain any NaN values. Let's take a look to see which columns have NaN values and determine how to handle them.

In [ ]:
df.count()[df.count() != len(df.index)]

If we drop the NaN values from the dataset which we'd lose 17.74% of our data which is too much and the fact that we have a smaller dataset.

In [ ]:
df['LotFrontage'].agg([min, max])

In [ ]:
df['LotFrontage'].hist(bins=10)

Based on this histogram we can see there's a few outlier values which would skew the mean so using the mean might not be the best choice in this case.

In [ ]:
la = df['LotArea'] ** 0.5
la.hist(bins=10)

In [ ]:
x = df['LotFrontage'] / df['LotArea']
x.hist()

We still have large outliers that would affect the average. What we'll do is just take the mean of the ratios under 0.015 and then multiply the lot area by this value to get the missing lot frontage.


But what is the median which isn't as adversely affected by the outliers?

In [ ]:
df['LotFrontage'].median()

Let's take a quick and dirty look at the amount of LotFrontage if we use the ratio method.

In [ ]:
df_copy = df.loc[df['LotFrontage'].isnull(), 'LotArea'].copy().to_frame()
df_copy['LotFrontage'] = 0.007 * df['LotArea']
df_copy

As we can see there are a lot of larger lots which might skew the data if we used the median method.

In [ ]:
df = impute_lot_frontage(df)
df.head(n=20)

In [ ]:
# Sale Price
from scipy.stats import boxcox

xt, maxlog = boxcox(df['SalePrice'])
print("lambda = {:g}".format(maxlog))

The Box-Cox test quickly and easily tells us if  we need to perform a transform on our data or not by telling us a lambda value:
* -1. is a reciprocal
* -.5 is a recriprocal square root
* 0.0 is a log transformation
* .5 is a square toot transform and
* 1.0 is no transform.

Since ours closest to 0 we need to perform a log transform on the sale price.

In [ ]:
df['SalePrice'] = np.log(df['SalePrice'])

In [ ]:
# Use a standard scaler to remove the mean and 
# make the standard deviation 1 for all of our features
# It will give us a range between 0 and 1
from sklearn.preprocessing import StandardScaler

# Using the defaults:
# - copy = True - Inplace / False doesn't always work
# - with_mean = True - May cause problems with a sparse matrix. We're going to try it as is and see how it fares
# - - Sparse matrix is a matrix where most feature values are 0
# - - - In our case we one hot encoded a lot of columns so we have a lot of 0 values
# - with_std = True - Scale with a single standard deviation
scaler = StandardScaler()
train_df = pd.read_csv(os.path.join('data','train.csv'))
test_df = pd.read_csv(os.path.join('data','test.csv'))
train_df = process_df(train_df, ohe_list)
test_df = process_df(test_df, ohe_list)

In [ ]:
train_df.shape

In [ ]:
test_df.shape

Initially the shape of the training and test sets do not align because of the One Hot Encoding might not have the same combinations.

In [ ]:
cols_train_not_test = [c for c in train_df.columns if c not in test_df.columns and c != 'SalePrice']
cols_train_not_test

In [ ]:
cols_test_not_train = [c for c in test_df.columns if c not in train_df.columns]
cols_test_not_train

Problem identified! We'll need to make sure the training and testing data sets have the same features.

In [ ]:
for c in cols_train_not_test:
    test_df[c] = 0
    
for c in cols_test_not_train:
    train_df[c] = 0

In [ ]:
test_df.shape

In [ ]:
train_df.shape

In [ ]:
# Which test data columns have NaNs?
null_columns = [col for col in test_df.columns if sum(test_df[col].isnull())]
null_columns

In [ ]:
# Set these each to 0 because they don't have the features described
for c in null_columns:
    test_df[c] = test_df[c].fillna(0)

<font color='red'>GOTCHA: We ran into the issue of absurd home prices (negative and positive * e^18) because we initially removed the sale price from the dataframe prior to scaling. This is why scaling ALL of your data point the same is very important. Refer to submissions/LinearRegression_submission_1.csv for the actual values.</font>

In [ ]:
# Remove SalePrice from dataframe before fit
# fit the remaining features
# scale x_train and x_test (transform)
# set y_train = SalePrice column

y_train = train_df['SalePrice'].copy()
train_df2 = train_df.drop('SalePrice', axis=1)


X_train = train_df2
X_test = test_df

#scaler.fit(train_df2)
#X_train = scaler.transform(train_df2)
#X_test = scaler.transform(test_df)


# Machine Learning Models!
a.k.a. the fun 20% rather than the annoying 80%.

Final Submission:
CSV File in the form of:
*ID, Predicted Sale Price*

Since we're predicting the home price and it's a continuous value Linear Regression model might be a good place to start.


In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
predicted_prices = lr.predict(X_test)

In [ ]:
# Revert log of predicted prices to determine 
# estimated sale price
predicted_prices = np.exp(predicted_prices)

In [ ]:
max(predicted_prices)

In [ ]:
plt.hist(predicted_prices)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfg = RandomForestRegressor()
rfg.fit(X_train, y_train)
predicted_prices = rfg.predict(X_test)
predicted_prices = np.exp(predicted_prices)

In [ ]:
plt.hist(predicted_prices)

In [ ]:
submission_df = pd.DataFrame({'Id':test_df['Id'], 'SalePrice':predicted_prices})
submission_df.to_csv(os.path.join('submissions', 'submission.csv'), index=False)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rfg, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
best_rf = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
best_rf.fit(X_train, y_train)

In [ ]:
best_rf.predict(X_test)

In [ ]:
predicted_prices = best_rf.predict(X_test)
predicted_prices = np.exp(predicted_prices)

In [ ]:
submission_df = pd.DataFrame({'Id':test_df['Id'], 'SalePrice':predicted_prices})
submission_df.to_csv(os.path.join('submissions', 'submission.csv'), index=False)

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train)
dtest = xgb.DMatrix(X_test)
#xgb_model = xgb.XGBRegressor(silent=False,
#                            )